In [1]:
import pandas as pd
import numpy as np
import re
import string
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidataVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [2]:
data = pd.read_csv("complaints.csv")

C:\Users\prana\AppData\Local\Temp\ipykernel_23752\1047253087.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("complaints.csv")


In [3]:
data.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided,Submitted via,Date sent to company,Company response to consumer,Timely response,Consumer disputed,Complaint ID
0,29-01-2025,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Account status incorrect,NaN,NaN,Experian Information Solutions Inc.,NY,12543,NaN,NaN,Web,29-01-2025,In progress,Yes,NaN,11825440
1,31-01-2025,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Account information incorrect,NaN,NaN,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",OH,43207,NaN,NaN,Web,31-01-2025,In progress,Yes,NaN,11844238
2,30-01-2025,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,Experian Information Solutions Inc.,NV,89148,NaN,NaN,Web,30-01-2025,In progress,Yes,NaN,11844247
3,29-01-2025,"Payday loan, title loan, personal loan, or adv...",Installment loan,Problem when making payments,NaN,NaN,NaN,TD BANK US HOLDING COMPANY,FL,33055,NaN,NaN,Phone,04-02-2025,In progress,Yes,NaN,11824206
4,28-01-2025,Mortgage,Conventional home mortgage,Applying for a mortgage or refinancing an exis...,Trying to communicate with the company to fix ...,NaN,NaN,"NEXA Mortgage, LLC",CA,95370,NaN,NaN,Phone,06-02-2025,In progress,Yes,NaN,11805339


In [4]:
data.tail()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided,Submitted via,Date sent to company,Company response to consumer,Timely response,Consumer disputed,Complaint ID
1048570,26-09-2023,Credit reporting or other personal consumer re...,Credit reporting,Problem with a company's investigation into an...,Their investigation did not fix an error on yo...,"Despite multiple written requests, the unverif...",Company has responded to the consumer and the ...,Experian Information Solutions Inc.,FL,342XX,NaN,Consent provided,Web,26-09-2023,Closed with explanation,Yes,NaN,7606237
1048571,26-09-2023,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",MI,48180,NaN,Consent not provided,Web,26-09-2023,Closed with explanation,Yes,NaN,7606239
1048572,21-12-2023,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,I firmly insist on the removal of these mislea...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,TX,76227,NaN,Consent provided,Web,21-12-2023,Closed with explanation,Yes,NaN,8043790
1048573,21-12-2023,Credit reporting or other personal consumer re...,Credit reporting,Problem with a company's investigation into an...,Their investigation did not fix an error on yo...,I am beyond frustrated with the poor handling ...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NY,10010,NaN,Consent provided,Web,21-12-2023,Closed with non-monetary relief,Yes,NaN,8032020
1048574,22-12-2023,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,15 USC 1681 a ( d ) ( 2 ) ( A ) ( i ) clearly ...,NaN,"EQUIFAX, INC.",IL,604XX,NaN,Consent provided,Web,22-12-2023,Closed with non-monetary relief,Yes,NaN,8047115


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 18 columns):
 #   Column                        Non-Null Count    Dtype 
---  ------                        --------------    ----- 
 0   Date received                 1048575 non-null  object
 1   Product                       1048575 non-null  object
 2   Sub-product                   1047701 non-null  object
 3   Issue                         1048575 non-null  object
 4   Sub-issue                     988638 non-null   object
 5   Consumer complaint narrative  134061 non-null   object
 6   Company public response       315505 non-null   object
 7   Company                       1048575 non-null  object
 8   State                         1046250 non-null  object
 9   ZIP code                      1048546 non-null  object
 10  Tags                          39998 non-null    object
 11  Consumer consent provided     440484 non-null   object
 12  Submitted via                 1048575 non-

In [6]:
data.describe(include="all")

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided,Submitted via,Date sent to company,Company response to consumer,Timely response,Consumer disputed,Complaint ID
count,1048575,1048575,1047701,1048575,988638,134061,315505,1048575,1046250,1048546,39998,440484,1048575,1048575,1048575,1048575,2317,1.048575e+06
unique,1920,20,79,156,262,86176,11,2484,60,16798,3,4,7,1939,8,2,2,NaN
top,17-01-2025,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,In accordance with the Fair Credit Reporting a...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,FL,XXXXX,Servicemember,Consent not provided,Web,17-01-2025,In progress,Yes,No,NaN
freq,32859,876490,874026,471219,290268,2239,301153,290556,146272,13155,30790,286394,1037450,28437,441848,1045995,1922,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.116892e+07
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.055498e+06
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.430000e+02
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.111255e+07
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.143130e+07
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.173564e+07


In [31]:
data.shape

(934197, 4)

In [32]:
data = data[['Issue', 'Product']]

category mapping

In [33]:
category_mapping = {
    "Credit reporting or other personal consumer reports": 0,
    "Debt collection": 1,
    "Payday loan, title loan, personal loan, or advance loan": 2,
    "Mortgage": 3
}
data['Category'] = data['Product'].map(category_mapping)
data.dropna(inplace=True)

Text Preprocessing

In [34]:
def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    text = re.sub(r'\d+', '', text)
    return text

data['Cleaned_Issue'] = data['Issue'].apply(clean_text)


In [35]:
X_train, X_test, y_train, y_test = train_test_split(data['Cleaned_Issue'], data['Category'], test_size=0.2, random_state=42)

Convert text to numeric features

In [36]:
vectorizer = TfidataVectorizer()
X_train_tfidata = vectorizer.fit_transform(X_train)
X_test_tfidata = vectorizer.transform(X_test)

LOGISTIC REGRESSION

In [37]:
LR = LogisticRegression()
LR.fit(X_train_tfidata, y_train)
y_pred = LR.predict(X_test_tfidata)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)



0.9994005566259901


In [38]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    175444
           1       1.00      1.00      1.00      9834
           2       0.99      0.88      0.93       453
           3       1.00      0.95      0.97      1109

    accuracy                           1.00    186840
   macro avg       1.00      0.96      0.98    186840
weighted avg       1.00      1.00      1.00    186840



In [47]:
def predict_category(sample_text):
    sample_text_cleaned = clean_text(sample_text)
    sample_vectorized = vectorizer.transform([sample_text_cleaned])
    prediction = LR.predict(sample_vectorized)[0]
    print(f"Logistic Regression Prediction: {prediction}")

# Example sample input
sample_input_LR = "Trouble during payment process"
predict_category(sample_input_LR)

Logistic Regression Prediction: 3


In [53]:
output_data = X_test.to_frame()
output_data['Actual'] = y_test.values
output_data['Predicted'] = y_pred
output_data.to_csv(f"predictions_LR.csv", index=False)
print("LR_predictions saved")


LR_predictions saved


NAIVE BAYES

In [40]:
NB = MultinomialNB()
NB.fit(X_train_tfidata, y_train)
y_pred = NB.predict(X_test_tfidata)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)



0.9991061871119674


In [41]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    175444
           1       1.00      1.00      1.00      9834
           2       1.00      0.75      0.86       453
           3       0.95      0.95      0.95      1109

    accuracy                           1.00    186840
   macro avg       0.99      0.93      0.95    186840
weighted avg       1.00      1.00      1.00    186840



In [48]:
def predict_category(sample_text):
    sample_text_cleaned = clean_text(sample_text)
    sample_vectorized = vectorizer.transform([sample_text_cleaned])
    prediction = NB.predict(sample_vectorized)[0]
    print(f"Naive Bayes Prediction: {prediction}")

# Example sample input
sample_input_NB = "Trouble during payment process"
predict_category(sample_input_NB)

Naive Bayes Prediction: 3


In [54]:
output_data = X_test.to_frame()
output_data['Actual'] = y_test.values
output_data['Predicted'] = y_pred
output_data.to_csv(f"predictions_NB.csv", index=False)
print("NB_predictions saved")

NB_predictions saved


RANDOM FOREST

In [42]:
RF = RandomForestClassifier(n_estimators=100, random_state=42)
RF.fit(X_train_tfidata, y_train)
y_pred = RF.predict(X_test_tfidata)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)


0.999421965317919


In [43]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    175444
           1       1.00      1.00      1.00      9834
           2       1.00      0.88      0.93       453
           3       1.00      0.95      0.98      1109

    accuracy                           1.00    186840
   macro avg       1.00      0.96      0.98    186840
weighted avg       1.00      1.00      1.00    186840



In [49]:
def predict_category(sample_text):
    sample_text_cleaned = clean_text(sample_text)
    sample_vectorized = vectorizer.transform([sample_text_cleaned])
    prediction = RF.predict(sample_vectorized)[0]
    print(f"Random Forest Prediction: {prediction}")

# Example sample input
sample_input_RF = "Trouble during payment process"
predict_category(sample_input_RF)

Random Forest Prediction: 3


In [55]:
output_data = X_test.to_frame()
output_data['Actual'] = y_test.values
output_data['Predicted'] = y_pred
output_data.to_csv(f"predictions_RF.csv", index=False)
print("RF_predictions saved")

RF_predictions saved


SUPPORT VECTOR MACHINE

In [44]:
SVM = SVC(kernel='linear')
SVM.fit(X_train_tfidata, y_train)
y_pred = SVM.predict(X_test_tfidata)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9994166131449368


In [45]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    175444
           1       1.00      1.00      1.00      9834
           2       1.00      0.88      0.93       453
           3       1.00      0.95      0.98      1109

    accuracy                           1.00    186840
   macro avg       1.00      0.96      0.98    186840
weighted avg       1.00      1.00      1.00    186840



In [51]:
def predict_category(sample_text):
    sample_text_cleaned = clean_text(sample_text)
    sample_vectorized = vectorizer.transform([sample_text_cleaned])
    prediction = SVM.predict(sample_vectorized)[0]
    print(f"Support Vector Machines Prediction: {prediction}")

# Example sample input
sample_input_SVM = "Trouble during payment process"
predict_category(sample_input_SVM)


Support Vector Machines Prediction: 3


In [56]:
output_data = X_test.to_frame()
output_data['Actual'] = y_test.values
output_data['Predicted'] = y_pred
output_data.to_csv(f"predictions_SVM.csv", index=False)
print("SVM_predictions saved")

SVM_predictions saved
